In [1]:
import pandas as pd
import re
import numpy as np


In [2]:
df=pd.read_csv("C:/Users/vrunda/NLP/FAKE-NEWS-CLASSIFIER/fake-news/train.csv")

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
# Droping nan values
df=df.dropna()

In [5]:
# get the independent features

X=df.drop('label',axis=1)


In [6]:
# dependent feature
y=df['label']

In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [8]:
# defining voculary size
voc_size=5000

# One Hot Representation

In [9]:
messages=X.copy()

In [10]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [11]:
messages.reset_index(inplace=True)

In [12]:
import nltk
from nltk.corpus import stopwords

In [14]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):

    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [15]:


onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr



[[1984, 1596, 652, 2548, 4656, 96, 4791, 3179, 3845, 275],
 [187, 545, 2044, 2787, 2226, 146, 214],
 [4545, 3967, 1088, 2192],
 [695, 1524, 396, 3659, 4418, 3824],
 [3109, 2226, 4959, 1962, 3057, 4008, 2226, 1711, 3561, 2120],
 [4573,
  4675,
  3020,
  4869,
  3607,
  2690,
  3549,
  4616,
  1013,
  2682,
  1525,
  1706,
  378,
  1017,
  214],
 [522, 2729, 3649, 3568, 2026, 4113, 1692, 23, 2178, 2825, 2448],
 [4855, 235, 2481, 2133, 4648, 568, 2690, 3441, 2178, 2825, 2448],
 [4985, 2763, 577, 3687, 5, 32, 1389, 2202, 2690, 2144],
 [4818, 2159, 3239, 879, 4709, 3788, 761, 1751],
 [3138, 4040, 1892, 1594, 3729, 608, 3618, 2185, 2861, 4702, 4625],
 [3659, 3655, 4656, 32, 2690, 4648],
 [346, 4586, 4010, 1245, 1635, 705, 2329, 795, 4289],
 [4989, 2107, 4813, 1599, 4006, 737, 2533, 2178, 2825, 2448],
 [1725, 2181, 3890, 4119, 2515, 2178, 2825, 2448],
 [3756, 3421, 3350, 262, 3282, 613, 2236, 4826, 1593, 1364],
 [3158, 3235, 545],
 [1274, 2571, 3609, 377, 2690, 1707, 2917, 214],
 [1094, 2842,

In [16]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3179 3845  275]
 [   0    0    0 ... 2226  146  214]
 [   0    0    0 ... 3967 1088 2192]
 ...
 [   0    0    0 ... 2178 2825 2448]
 [   0    0    0 ... 3597   59  489]
 [   0    0    0 ... 1018 4413 3132]]


In [17]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [20]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 24ms/step - loss: 0.3340 - accuracy: 0.8393 - val_loss: 0.1904 - val_accuracy: 0.9188
Epoch 2/10
192/192 [==============================] - 4s 22ms/step - loss: 0.1356 - accuracy: 0.9476 - val_loss: 0.2252 - val_accuracy: 0.9208
Epoch 3/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0925 - accuracy: 0.9663 - val_loss: 0.2284 - val_accuracy: 0.9057
Epoch 4/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0724 - accuracy: 0.9758 - val_loss: 0.2558 - val_accuracy: 0.9133
Epoch 5/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0485 - accuracy: 0.9839 - val_loss: 0.3115 - val_accuracy: 0.9181
Epoch 6/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0310 - accuracy: 0.9902 - val_loss: 0.3733 - val_accuracy: 0.9110
Epoch 7/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0212 - accuracy: 0.9945 - val_loss: 0.4578 - val_accuracy: 0.9143

# Adding dropout

In [ ]:
#from tensorflow.keras.layers import Dropout
## Creating model
#embedding_vector_features=40
#model=Sequential()
#model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
#model.add(Dropout(0.3))
#model.add(LSTM(100))
#model.add(Dropout(0.3))
#model.add(Dense(1,activation='sigmoid'))
#model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Performance metrics and accuracy

In [21]:
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [22]:
from sklearn.metrics import confusion_matrix

In [23]:
confusion_matrix(y_test,y_pred)

array([[3201,  218],
       [ 388, 2228]], dtype=int64)

In [24]:


from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)



0.8995857497928749